# Linear Regression

## Linear Regression

Hàm mất mát của Linear Regression là:
$~~~\mathcal{L}(\mathbf{w,b}) = \frac{1}{2N}||\mathbf{y - (\bar{X}w} +b)||_2^2$

Đạo hàm của hàm mất mát là:
$~~~\nabla_{\mathbf{w}}\mathcal{L}(\mathbf{w,b}) = \frac{1}{N}\mathbf{\bar{X}}^T \mathbf{((\bar{X}w + b) - y)} ~~~~~(1)$

$~~~\nabla_{\mathbf{b}}\mathcal{L}(\mathbf{w,b}) = \frac{1}{N}\mathbf((\bar{X}w + b) - y) ~~~~~(2)$

Cập nhật gradient cho một biến: $~~~~~x_{t+1} = x_{t} - \eta f’(x_{t})$

=> Cập nhật gradient cho Weight và bias:
    $\mathbf{w} \leftarrow \mathbf{w} - \alpha\nabla_{w}~~~$;
    $~~~~~\mathbf{b} \leftarrow \mathbf{b} - \alpha\nabla_{b}$

## Regularization

instead of simply aiming to minimize loss (empirical risk minimization):
$$\text{minimize(Loss(Data|Model))}$$

we'll now minimize loss+complexity, which is called **structural risk minimization**:
$$\text{minimize(Loss(Data|Model)} + \lambda \text{ complexity(Model))}$$

Our training optimization algorithm is now a function of two terms: the loss term, which measures how well the model fits the data, and the regularization term, which measures model complexity.

[Why L1 norm for sparse models](https://stats.stackexchange.com/a/159379/388752)

### $L_1\text{ - LASSO Regularization}$

$$L_1\text{ regularization term} = R(\mathbf{w}) = ||\boldsymbol w||_1 = \Sigma_i |w_i| $$

Graident: $$\frac{dL_1(w)}{dw} = sign(w) \text{ where } sign(w) = (\frac{w_1}{|w_1|}, \frac{w_2}{|w_2|}, \dots, \frac{w_m}{|w_m|})$$

Notice that for $L_1$, the gradient is either 1 or -1, except for when $w_i$=0. That means that L1-regularization will move any weight towards 0 with the same step size, regardless the weight's value

<img src="utils/img/regularization/l1.png">

<img src="utils/img/regularization/l1_grad.png" width=300 height=300>

### $L_2\text{ - Ridge Regularization}$

$$L_2\text{ regularization term} = R(\mathbf{w}) = ||\boldsymbol w||_2^2 = {w_1^2 + w_2^2 + ... + w_n^2}$$

So in conclude, the loss regularized function:
$$J_{\text{reg}} = \frac{1}{2} \|\mathbf{y} - \mathbf{Xw}\|_2^2 + \lambda \|\mathbf{w}\|_2^2$$

=> gradient: 
$$\frac{\partial J_{\text{reg}} }{\partial \mathbf{w}} = \frac{\partial J}{\partial \mathbf{w}} + \lambda \mathbf{w}$$



### Dicussion

[When use L1, L2?](https://machinelearningcoban.com/2017/03/04/overfitting/#comment-3310034656)

> Q
- Tại sao Regularization lạ giảm được overfitting ?
- Khi nào thì chúng ta dùng L1 và L2 . Em cảm ơn .

> A

Khi có regularization thì các hệ số 'bị ép' vào khuôn khổ chứ không được tự do bay nhảy nữa.

L2 giúp cho các hệ số nhỏ, không được quá lớn. L1 giúp cho hầu hết các hệ số bằng 0.
Thường thì người ta dùng L2 vì đạo hàm đẹp. L1 không có đạo hàm tại 0 và ít được sử dụng hơn. Khi thực hành, có thể chọn L2 trước xem kq thế nào, nếu tệ thì thử sang L1.

### Implementation

In [5]:
import numpy as np

class LinearRegression():
    # Constructor
    def __init__(self, learning_rate=0.01, epochs=20, regularize=None, lamda=0.01):
        self.lr = learning_rate
        self.epochs = epochs
        self.w = None
        self.b = None
        self.regularize = regularize
        self.lamda = lamda
    # Fit
    def fit(self,X, y):
        # init parameters
        n_samples, n_features = X.shape
        self.w = np.random.rand(n_features)
        self.b = 0
        threshold = 0.001
        
        w_best = self.w.copy()
        # gradient descent
        for _ in range(self.epochs):
            
            y_predicted = self.predict(X)
            dW = 1/n_samples*(X.T@(y_predicted-y))
            dB = 1/n_samples*(y_predicted-y)
            self.w -= self.lr*dW
            self.b -= self.lr*np.sum(dB)
            # regularization
            if self.regularize == 'l1':
                self.w -= 1/n_samples*self.lamda*np.sign(self.w)
            elif self.regularize == 'l2':
                self.w -= 1/n_samples*self.lamda*self.w

            if np.allclose(self.w, w_best, threshold): break
            else: w_best = self.w.copy()
        return self
    
    # Predict 
    def predict(self, X):
        return X@self.w +self.b
    
    # Evaluate by RMSE
    def rmse(self,y, y_predicted):
        return 1/(2*len(y))*np.sqrt(np.sum((y-y_predicted)**2))

Train, Test

In [6]:
from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [14]:
diabetes = datasets.load_diabetes()

data = diabetes.data
target = diabetes.target
features=diabetes.feature_names

# train test split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

# just for understand data
df = pd.DataFrame(X_train, columns=features)
df['target'] = y_train

In [13]:
model = LinearRegression(regularize='l1', lamda=0.01)
X_train = StandardScaler().fit_transform(X_train)
model.fit(X_train,y_train)

y_predicted = model.predict(X_test)

model.rmse(y_test, y_predicted)

6.026617043248765

In [12]:
from sklearn.metrics import mean_squared_error
model_lib = linear_model.LinearRegression().fit(X_train,y_train)

y_predicted_lib = model_lib.predict(X_test)

model.rmse(y_test, y_predicted_lib)

3.060569566721311

# Cross Validation

In [53]:
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
# for i, (train_index, test_index) in enumerate(kf):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

## Manual

In [15]:
n_folds = 5
models = []
models_rmse = []
n_samples,n_features = data.shape
for i in range(n_folds):
    test_index = np.array([index for index in range(i*(n_samples // n_folds),(i+1)*(n_samples // n_folds) if (i+1)*(n_samples // n_folds) < data.shape[0] else data.shape[0])])
    train_index = np.array(list(set([j for j in range(n_samples)]) - set(test_index)))
    model = LinearRegression()
    model.fit(data[train_index],target[train_index])

    y_predicted = model.predict(data[test_index])
    rmse = model.rmse(target[test_index], y_predicted)
    models.append(model)
    models_rmse.append(rmse)

In [16]:
np.average(np.array(models_rmse))

7.783711084159646